In [67]:
# imports
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import sklearn as sk
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from tf.keras.optimizers import SGD

import seaborn as sn
import matplotlib.pyplot as plt

print("TF version {}".format(tf.__version__))
print("sklearn version {}".format(sk.__version__))
print("numpy version {}".format(np.__version__))
print("pandas version {}".format(pd.__version__))
print("seaborn version {}".format(sn.__version__))

print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")


TF version 2.1.0
sklearn version 0.22.2.post1
numpy version 1.18.1
pandas version 1.0.3
seaborn version 0.10.1
Eager mode:  True
GPU is NOT AVAILABLE


In [2]:
# load the training data
titanic_df = pd.read_csv("../../../../Datasets/Kaggle/Competitions/Titanic/train.csv")

titanic_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
# gets stats on the data
titanic_df.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
# look at data
titanic_df.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
# drop from train
# passengerId, name, ticket
train_df = titanic_df.filter(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], axis=1)
label_df = titanic_df.filter(['Survived'], axis=1)


# get info on train df
train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [10]:
# look at the data
train_df.head(5)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
560,3,male,NaN,0,0,7.7500,Q
386,3,male,1.0,5,2,46.9000,S
260,3,male,NaN,0,0,7.7500,Q
212,3,male,22.0,0,0,7.2500,S
164,3,male,1.0,4,1,39.6875,S


In [11]:
# look at pClass (passenger class?)
train_df['Pclass'].value_counts().head()


3    491
1    216
2    184
Name: Pclass, dtype: int64

In [13]:
# add pclass one hot fields
train_dummies_df = pd.get_dummies(train_df)

train_dummies_df.head(5)


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
560,3,NaN,0,0,7.7500,0,1,0,1,0
386,3,1.0,5,2,46.9000,0,1,0,0,1
260,3,NaN,0,0,7.7500,0,1,0,1,0
212,3,22.0,0,0,7.2500,0,1,0,0,1
164,3,1.0,4,1,39.6875,0,1,0,0,1


In [19]:
pclass_dummies_df = pd.get_dummies(train_df, columns=['Pclass', 'SibSp', 'Sex', 'Parch', 'Embarked'])

pclass_dummies_df.head(5)

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
560,NaN,7.7500,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
386,1.0,46.9000,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
260,NaN,7.7500,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
212,22.0,7.2500,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
164,1.0,39.6875,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [20]:
pclass_dummies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 560 to 258
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         714 non-null    float64
 1   Fare        891 non-null    float64
 2   Pclass_1    891 non-null    uint8  
 3   Pclass_2    891 non-null    uint8  
 4   Pclass_3    891 non-null    uint8  
 5   SibSp_0     891 non-null    uint8  
 6   SibSp_1     891 non-null    uint8  
 7   SibSp_2     891 non-null    uint8  
 8   SibSp_3     891 non-null    uint8  
 9   SibSp_4     891 non-null    uint8  
 10  SibSp_5     891 non-null    uint8  
 11  SibSp_8     891 non-null    uint8  
 12  Sex_female  891 non-null    uint8  
 13  Sex_male    891 non-null    uint8  
 14  Parch_0     891 non-null    uint8  
 15  Parch_1     891 non-null    uint8  
 16  Parch_2     891 non-null    uint8  
 17  Parch_3     891 non-null    uint8  
 18  Parch_4     891 non-null    uint8  
 19  Parch_5     891 non-null   

In [54]:
# scale fare and age columns
scaler = MinMaxScaler()
pclass_dummies_df[['Fare', 'Age']] = scaler.fit_transform(pclass_dummies_df[['Fare', 'Age']])

pclass_dummies_df.head(5)

,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
560,NaN,0.015127,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
386,0.007288,0.091543,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
260,NaN,0.015127,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
212,0.271174,0.014151,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
164,0.007288,0.077465,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [55]:
# shuffle the data
train_df, label_df = shuffle(pclass_dummies_df, label_df, random_state=2)

print("the train dataset is of shape {} and train label of shape{}".format(train_df.shape, label_df.shape))

the train dataset is of shape (891, 24) and train label of shape(891, 1)


In [56]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(train_df, label_df, test_size=0.2)

print("the train dataset is of shape {} and train label of shape{}".format(X_train.shape, y_train.shape))
print("the test dataset is of shape {} and test label of shape{}".format(X_test.shape, y_test.shape))


the train dataset is of shape (712, 24) and train label of shape(712, 1)
the test dataset is of shape (179, 24) and test label of shape(179, 1)


In [74]:
# build a model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(24,)),
    # tf.keras.layers.Dense(32, activation='relu'),
    # tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss='hinge', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='squared_hinge', optimizer='adam', metrics=['accuracy'])

# model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# opt = SGD(lr=0.01, momentum=0.9)
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


# summarize the model
model.summary()


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_50 (Dense)             (None, 32)                800       
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 33        
Total params: 833
Trainable params: 833
Non-trainable params: 0
_________________________________________________________________


In [75]:
# fit the model
number_epochs = 20

model.fit(X_train, y_train, epochs=number_epochs, batch_size=20, validation_data=[X_test, y_test])

Train on 712 samples, validate on 179 samples
Epoch 1/20
712/712 [==============================] - 0s 469us/sample - loss: nan - accuracy: 0.6194 - val_loss: nan - val_accuracy: 0.6089
Epoch 2/20
712/712 [==============================] - 0s 58us/sample - loss: nan - accuracy: 0.6180 - val_loss: nan - val_accuracy: 0.6089
Epoch 3/20
712/712 [==============================] - 0s 58us/sample - loss: nan - accuracy: 0.6180 - val_loss: nan - val_accuracy: 0.6089
Epoch 4/20
712/712 [==============================] - 0s 59us/sample - loss: nan - accuracy: 0.6180 - val_loss: nan - val_accuracy: 0.6089
Epoch 5/20
712/712 [==============================] - 0s 59us/sample - loss: nan - accuracy: 0.6180 - val_loss: nan - val_accuracy: 0.6089
Epoch 6/20
712/712 [==============================] - 0s 58us/sample - loss: nan - accuracy: 0.6180 - val_loss: nan - val_accuracy: 0.6089
Epoch 7/20
712/712 [==============================] - 0s 59us/sample - loss: nan - accuracy: 0.6180 - val_loss: nan - v